In [21]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

data = pd.read_csv('breast-cancer.data', header=0) 
data.head().T

,0,1,2,3,4
class,no-recurrence-events,no-recurrence-events,no-recurrence-events,no-recurrence-events,no-recurrence-events
age,30-39,40-49,40-49,60-69,40-49
menopause,premeno,premeno,premeno,ge40,premeno
tumor-size,30-34,20-24,20-24,15-19,0-4
inv-nodes,0-2,0-2,0-2,0-2,0-2
node-caps,no,no,no,no,no
deg-malig,3,2,2,2,2
breast,left,right,left,right,right
breast-quad,left_low,right_up,left_low,left_up,right_low
irradiat,no,no,no,no,no


In [58]:
feature_cols = ['class', 'age', 'menopause', 'tumor-size', 'inv-nodes', 'node-caps', 'deg-malig', 'breast', 'breast-quad']
X = data.loc[:, feature_cols]
y = data.loc[:, 'irradiat']

X = data.iloc[:, 0:-1]
y = data.iloc[:, -1]
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [59]:
counts = [X_train.shape[0], X_valid.shape[0], X_test.shape[0]]
print("Rozmiar zbioru treningowego:\t", counts[0])
print("Rozmiar zbioru walidacyjnego:\t", counts[1])
print("Rozmiar zbioru testowego:\t", counts[2])

Rozmiar zbioru treningowego:	 160
Rozmiar zbioru walidacyjnego:	 54
Rozmiar zbioru testowego:	 72
